<a href="https://colab.research.google.com/github/IndraniMandal/CSC310-S20/blob/master/Missing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data cleaning 

The original COVID-19 dataset was obtained from [Kaggle](https://www.kaggle.com/code/evanepstein/europe-covid-19-cases-exploratory-analysis/data). Some values were removed to create a dataset for this lesson.

Access the messy COVID-19 dataset from my [Github Repo ](https://https://github.com/IndraniMandal/ds-assets/blob/main/assets/messy_covid19_southamerica%20-%20covid19_southamerica.csv)

#Load Data

In [1]:
import pandas as pd, numpy as np 
# import csv file as a pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/IndraniMandal/ds-assets/main/assets/messy_covid19_southamerica%20-%20covid19_southamerica.csv')

In [ ]:
df.head()


,ObservationDate,Country_Region,Province_State,Confirmed,Deaths,Recovered,Active
0,2020-01-23,Mexico,NaN,0.0,0,0,0.0
1,2020-01-23,Brazil,NaN,0.0,0,0,0.0
2,2020-01-23,Colombia,NaN,0.0,0,0,0.0
3,2020-02-26,Brazil,NaN,1.0,0,0,1.0
4,2020-02-26,Brazil,NaN,1.0,0,0,1.0


# Missing values
Let's check if the data has any missing values. If so, how can we handle it?

In [2]:
df.isnull().sum()

ObservationDate       0
Country_Region        0
Province_State     5989
Confirmed             1
Deaths                3
Recovered             2
Active               26
dtype: int64

We notice the Province_State column has alot of missing values. 

# Data Imputation
There are different ways of handling missing data. 
## Imputation Using (Mean/Median) Values
This works by calculating the mean/median of the non-missing values in a column and then replacing the missing values within each column separately and independently from the others. It can only be used with numeric data.
We are going to replace the missing values by the mean of this column.

In [3]:
df['Confirmed'].fillna(df['Confirmed'].mean(), inplace=True)#replaces NaN's with the mean
df.Confirmed[df.Confirmed.isnull()]#check to make sure there are no NaNs left

Series([], Name: Confirmed, dtype: float64)

##Imputation Using (Most Frequent) or (Zero/Constant) Values
Most Frequent is another statistical strategy to impute missing values and YES!! It works with categorical features (strings or numerical representations) by replacing missing data with the most frequent values within each column.

In [4]:
df['Deaths'].fillna(0, inplace=True)#replaces NaN's with the median
df.Deaths[df.Deaths.isnull()]

Series([], Name: Deaths, dtype: object)

##Imputation Using k-NN
The k nearest neighbours is an algorithm that is used for simple classification. The algorithm uses ‘feature similarity’ to predict the values of any new data points. This means that the new point is assigned a value based on how closely it resembles the points in the training set. This can be very useful in making predictions about the missing values by finding the k’s closest neighbours to the observation with missing data and then imputing them based on the non-missing values in the neighbourhood.

Imputation doesn't work for non-numeric columns. Any column that is supposed to be numeric but is not should be converted to numeric. 

In [10]:
df['Recovered'] = pd.to_numeric(df['Recovered'], errors='coerce')

In [15]:
v = df.loc[: ,'Recovered':'Active']

In [16]:
v

,Recovered,Active
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,1.0
4,0.0,1.0
...,...,...
23145,64.0,0.0
23146,4667.0,67.0
23147,2275.0,2039.0
23148,1755.0,208.0


In [17]:
# import the SimpleImputer class
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
clean_df = imputer.fit_transform(v)

In [18]:
clean_df

array([[    0.,     0.],
       [    0.,     0.],
       [    0.,     0.],
       ...,
       [ 2275.,  2039.],
       [ 1755.,   208.],
       [63346.,  9568.]])

The transformed data is no longer a dataframe, it's a numpy array. We have to convert it back to a dataframe and concat it with the non-numeric columns.

In [19]:
clean_df = pd.DataFrame(clean_df, columns= ['Recovered', 'Active'])
clean_df.head()

,Recovered,Active
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,1.0
4,0.0,1.0


The non-numeric columns were the `ObservationDate` and `Country_Region`.

In [21]:
df.loc[ : ,'ObservationDate': 'Deaths']

,ObservationDate,Country_Region,Province_State,Confirmed,Deaths
0,2020-01-23,Mexico,NaN,0.0,0
1,2020-01-23,Brazil,NaN,0.0,0
2,2020-01-23,Colombia,NaN,0.0,0
3,2020-02-26,Brazil,NaN,1.0,0
4,2020-02-26,Brazil,NaN,1.0,0
...,...,...,...,...,...
23145,2020-09-28,Saint Vincent and the Grenadines,NaN,64.0,0
23146,2020-09-28,Suriname,NaN,4836.0,102
23147,2020-09-28,Trinidad and Tobago,NaN,4386.0,72
23148,2020-09-28,Uruguay,NaN,2010.0,47


Let's concatinate the data along the columns(`axis =1`).

In [22]:
transformed_df = pd.concat([df.loc[ : ,'ObservationDate': 'Deaths'],clean_df ], axis=1)
transformed_df

,ObservationDate,Country_Region,Province_State,Confirmed,Deaths,Recovered,Active
0,2020-01-23,Mexico,NaN,0.0,0,0.0,0.0
1,2020-01-23,Brazil,NaN,0.0,0,0.0,0.0
2,2020-01-23,Colombia,NaN,0.0,0,0.0,0.0
3,2020-02-26,Brazil,NaN,1.0,0,0.0,1.0
4,2020-02-26,Brazil,NaN,1.0,0,0.0,1.0
...,...,...,...,...,...,...,...
23145,2020-09-28,Saint Vincent and the Grenadines,NaN,64.0,0,64.0,0.0
23146,2020-09-28,Suriname,NaN,4836.0,102,4667.0,67.0
23147,2020-09-28,Trinidad and Tobago,NaN,4386.0,72,2275.0,2039.0
23148,2020-09-28,Uruguay,NaN,2010.0,47,1755.0,208.0


#Groupby
Minimun number of Active cases

In [23]:
df=transformed_df.groupby('Country_Region')['Active'].min()
df

Country_Region
Antigua and Barbuda                      0.0
Argentina                                1.0
Bahamas                                  0.0
Barbados                                 0.0
Belize                                   0.0
Bolivia                                  2.0
Brazil                             -283952.0
Chile                              -232210.0
Colombia                            -10074.0
Costa Rica                               1.0
Cuba                                     3.0
Dominica                                 0.0
Dominican Republic                       1.0
Ecuador                                  6.0
El Salvador                              1.0
French Guiana                            5.0
Grenada                                  0.0
Guadeloupe                               1.0
Guatemala                                1.0
Guyana                                   0.0
Haiti                                    2.0
Honduras                                